In [ ]:
import numpy as np
import pandas as pd
import random
import networkx as nx
from IPython.display import Image
import matplotlib.pyplot as plt
import nltk
from random import choice
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sources =[]
sinks = []
neighbours = []
f = open('/content/drive/My Drive/Colab Notebooks/train.txt',encoding ='utf-8')
lines = f.readlines()
for line in lines:
    data = [int(i) for i in line.split()]
    source = data[0]
    sources.append(source)
    data.pop(0)
    neighbours.append(data)
    sinks.extend(data)

train = {}
notEmpNodes = []
for i in range(len(sources)):
    train[sources[i]] = neighbours[i]
    if not neighbours[i] == []:
        notEmpNodes.append(sources[i])

In [ ]:
positive = []
negative = []
for a in range(10000):
    node1 = choice(sources)
    node2 = choice(sinks)
    while ((node1,node2) in negative) or (node2 in train[node1]):
        node1 = choice(sources)
        node2 = choice(sinks)
    negative.append((node1,node2))
for a in range(10000):
    node1 = choice(notEmpNodes)
    node2 = choice(train[node1])
    while (node1,node2) in positive:
        node1 = choice(notEmpNodes)
        node2 = choice(train[node1])
    positive.append((node1,node2))

In [ ]:
tes_posi = []
tes_nega = []
for a in range(2000):
    node1 = choice(sources)
    node2 = choice(sinks)
    while ((node1,node2) in tes_nega) or (node2 in train[node1]):
        node1 = choice(sources)
        node2 = choice(sinks)
    tes_nega.append((node1,node2))
for a in range(2000):
    node1 = choice(notEmpNodes)
    node2 = choice(train[node1])
    while (node1,node2) in tes_posi:
        node1 = choice(notEmpNodes)
        node2 = choice(train[node1])
    tes_posi.append((node1,node2))

In [ ]:
G = nx.Graph(train)

In [ ]:
def get_features(G,edges):
    jcs = []
    aas = []
    ras = []
    pas = []
    deg = []
    for edge in edges:
        
            jc = list(nx.jaccard_coefficient(G,[edge]))
            for source,sink,jcscore in jc:
                jcs.append(jcscore)
    
    
            aa = list(nx.adamic_adar_index(G,[edge]))
            for source,sink,aascore in aa:
                aas.append(aascore)

    
            ra = list(nx.resource_allocation_index(G,[edge]))
            for source,sink,rascore in ra:
                ras.append(rascore)
    
    
            pa = list(nx.preferential_attachment(G,[edge]))
            for source,sink,pascore in ra:
                pas.append(pascore)

            de = G.degree(edge[0]) + G.degree(edge[1])
            deg.append(de)
        
    return jcs, aas, ras, pas, de

In [ ]:
pos_jc, pos_aa, pos_ra, pos_pa, pos_de = get_features(G,positive)
neg_jc, neg_aa, neg_ra, neg_pa, neg_de = get_features(G,negative)

In [ ]:
pos_label = [1] * len(positive)
neg_label = [0] * len(negative)

In [ ]:
train_jc = pos_jc + neg_jc
train_aa = pos_aa + neg_aa
train_ra = pos_ra + neg_ra
train_pa = pos_pa + neg_pa
train_de = pos_de + neg_de
train_label = np.array(pos_label + neg_label)
trainset = pd.DataFrame()
trainset['JC'] = train_jc
trainset['AA'] = train_aa
trainset['RA'] = train_ra
trainset['PA'] = train_pa
trainset['DE'] = train_de

In [ ]:
jc_tes_pos, aa_tes_pos, ra_tes_pos, pa_tes_pos, de_tes_pos = get_features(G,tes_posi)
jc_tes_neg, aa_tes_neg, ra_tes_neg, pa_tes_neg, de_tes_neg = get_features(G,tes_nega)

In [ ]:
tes_pos_label = [1] * len(tes_posi)
tes_neg_label = [0] * len(tes_nega)
tes_label = np.array(tes_pos_label + tes_neg_label)

In [ ]:
jc_tes = jc_tes_pos + jc_tes_neg
aa_tes = aa_tes_pos + aa_tes_neg
ra_tes = ra_tes_pos + ra_tes_neg
pa_tes = pa_tes_pos + pa_tes_neg
de_tes = de_tes_pos + de_tes_neg
testset = pd.DataFrame()
testset['JC'] = jc_tes
testset['AA'] = aa_tes
testset['RA'] = ra_tes
testset['PA'] = pa_tes 
testset['DE'] = de_tes

In [ ]:
np.save('positive.npy',np.array(positive))
np.save('negative.npy',np.array(negative))
np.save('train_label.npy',np.array(train_label))

In [ ]:
np.save('train_test_positive.npy',np.array(tes_posi))
np.save('train_test_negative.npy',np.array(tes_nega))
np.save('tes_label.npy',np.array(tes_label))

In [ ]:
trainset.to_csv("trainset.csv",index = False)
testset.to_csv("testset.csv",index = False)

In [ ]:
t = open('/content/drive/My Drive/Colab Notebooks/test-public.txt')
test = []
tlines = t.readlines()
id = []
for line in tlines[1:]:
    edge = line.split()
    test.append((int(edge[1]),int(edge[2])))
    id.append(edge[0])

In [ ]:
tes_jc, tes_aa, tes_ra, tes_pa, tes_de = get_features(G,test)

In [ ]:
test_final = pd.DataFrame()
test_final['JC'] = tes_jc
test_final['AA'] = tes_aa
test_final['RA'] = tes_ra
test_final['PA'] = tes_pa
test_final['DE'] = tes_de

In [ ]:
test_final.to_csv("test_final.csv",index = False)